In [1]:
#libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
import netCDF4
import xarray as xr
import pickle 

## Inputs


Dimensions specified below

$\textit{m}$: length of the measurement vector

$\textit{n}$: length of the state (parameter) vector

$\textit{r}$: length of the model error vector


To perform this ICA, you'll need a few things:

1. Jacobian matrix, defined as the $ K_{j,i}(\textbf{x}) = \frac{\partial F_i (\textbf{x})}{\partial x_j}$ where $\textit{i}$ has length $\textit{m}$ and $\textit{j}$ has length $\textit{n}$. $F(\textbf{x})$ is the forward model at the state defined by $\textbf{x}$.

    jac [n x m]
    
2. Model error Jacobian matrix, defined as the $ K_{b,u,i}(\textbf{x}) = \frac{\partial F_i (\textbf{x})}{\partial b_u}$ where $\textit{i}$ has length $\textit{m}$ and $\textit{u}$ has length $\textit{r}$. $F(\textbf{x})$ is the forward model at the state defined by $\textbf{x}$. Note there are options in the Rodgers function to omit consideration of model errors if these are not known.

    jac_me [r x m]
    
3. Measurement error coveriance matrix, $S_{\epsilon}$. This can be specificed as either an $\textit{m}$ length vector of sigma squared measurement uncertainties or an $\textit{m} x \textit{m}$ full covariance matrix. In the former case, it is assumed that measurement uncertainty is uncorrelated. 

    err [m x m] or [m]   

4. Model error coveriance matrix, $S_{b}$. This can be specificed as either an $\textit{r}$ length vector of sigma squared measurement uncertainties or an $\textit{r} x \textit{r}$ full covariance matrix. In the former case, it is assumed that measurement uncertainty is uncorrelated. Note there are options below to omit consideration of model errors if these are not known.

    err_me [m x m] or [m]   
    
5. A priori error coveriance matrix, $S_{a}$. This can be specificed as either an $\textit{n}$ length vector of sigma squared measurement uncertainties or an $\textit{n} x \textit{n}$ full covariance matrix. In the former case, it is assumed that measurement uncertainty is uncorrelated. 

    ap [n x n] or [n]   
    
6. If calculating the detection probability: the value of the parameter in question

    mu [scalar]
    
        

# Outputs


### from rodgers()
1. Error covariance matrix, $\hat{S}$.

    S_hat [n x n]
    
2. Shannon Information Content, $SIC$.

    SIC [scalar]
    
3. Averaging kernel matrix, $A$. 

    AvgK [n x n]

4. Degrees of Freedom for Signal, $DFS$. 

    DFS [scalar]  

    
### from detect_prob()
1. Probability of detection, $P_d$. Note this requires inputs mu (parameter value in question) and sigma ($\sqrt{\hat{S}_{p,p}}$ where $p$ is the parameter index.)

    Pd [scalar]
    
    Pd_pcnt_str [percentage as string]


## from Amir:
    Relative:
σ_sys = np.array([0.01587401, 0.01244266, 0.00938955, 0.01092051, 0.00302288,
                  0.00544271, 0.00999068, 0.01467843, 0.0080387, 0.00944394,
                  0.0193447, 0.0224503, 0.02386379])

Absolute:
σ_sys = np.array([2.80453033e-03, 1.52152435e-03, 1.04028473e-03, 9.98138237e-04,
                  1.12356970e-04, 2.68196657e-04, 4.69710954e-04, 4.32004600e-04,
                  2.49862337e-04, 2.53730871e-04, 8.83851738e-05, 1.47447273e-04,
                  1.57220696e-04])

This is based on real modisa data using our OE algorithm, where we use the full spectrum to optimize things, which means that these values work better when you use all bands in MODIS. The systematic uncertainty dominates the random noise, so I feel comfortable ignoring the random component.

In [2]:
#function to calculate the parameter error covariance matrix.

#input Jacobian, K, [n x m], error covariance matrix Se, [m x m] and a priori matrix Sa, [n x n]
#model_error and model_error_jacobian describe model uncertainty - that is, parameterized uncertainty 
#and its jacobian
def rodgers(jac, err, ap, model_error={}, model_error_jacobian={}): 
    
        #check if error covariance matrix is square, or just diagonal values. If latter make full matrix
    if err.ndim == 1:
        ln=np.shape(err)
        err2d = np.zeros((ln[0], ln[0]))
        np.fill_diagonal(err2d, err)
        err=err2d

        #check if a priori covariance matrix is square, or just diagonal values. If latter make full matrix
    if ap.ndim == 1:
        ln=np.shape(ap)
        ap2d = np.zeros((ln[0], ln[0]))
        np.fill_diagonal(ap2d, ap)
        ap=ap2d        
            
        #section to verify compatable dimensions ------------------------------------------------------
    sh_jac = np.shape(jac)
    sh_err = np.shape(err)
    sh_ap = np.shape(ap)
    
    n_dim = sh_jac[0]
    m_dim = sh_jac[1]
    
    if not((sh_err[0] == sh_err[1]) and (sh_ap[0] == sh_ap[1])):
        print('ERROR: error covariance matrix or a priori matrix are not square')
        print('Error covariance matrix dimensions')
        print(sh_err)
        print('A priori matrix dimensions')
        print(sh_ap)
        return -1, -1, -1, -1
    
    if not(sh_jac[0] == sh_ap[0]):
        print('ERROR: n dimensions inconsistent, should be Jacobian [n x m]; a priori [n x n]')
        print('Jacobian matrix dimensions')
        print(sh_jac)
        print('A priori matrix dimensions')
        print(sh_ap)
        return -1, -1, -1, -1
    
    if not(sh_jac[1] == sh_err[0]):
        print('ERROR: m dimensions inconsistent, should be Jacobian [n x m]; error covariance [m x m]')
        print('Jacobian matrix dimensions')
        print(sh_jac)
        print('Error covariance matrix dimensions')
        print(sh_err)
        return -1, -1, -1, -1
        
    #section to generate model derived error -------------------------------------------------------
    
    if len(model_error) > 0:
        me=model_error
        jac_me=model_error_jacobian
        
        ln_me=np.shape(me)
        errme_2d = np.zeros((ln_me[0], ln_me[0]))
        np.fill_diagonal(errme_2d, me)
        err_me=errme_2d
    
        jac_me_t=np.transpose(jac_me)      
    
        JacmetMeJacme = np.matmul(jac_me_t,np.matmul(err_me,jac_me))
        err = err + JacmetMeJacme
    
        #perform inverse and matrix multiplication calculations ----------------------------------------
    jac_t=np.transpose(jac) #transpose of Jacobian (KT)
    
    try: 
        err_i=np.linalg.inv(err) #inverse of error covariance matrix (Se-1)
    except:
        print("ERROR: problem inverting error covariance matrix")
        return -1, -1, -1, -1
    
    try: 
        ap_i=np.linalg.inv(ap) #inverse of a priori error covariance matrix
    except:
        print("ERROR: problem inverting a priori covariance matrix")
        return -1, -1, -1, -1

    KtSK = np.matmul(jac,np.matmul(err_i,jac_t)) #calcuates KT Se-1 K

    try: 
        S_hat = np.linalg.inv(KtSK+ap_i) #calculate the inverse of (above + Sa-1)
    except:
        print("ERROR: problem inverting retrieval error covariance matrix")
        return -1, -1, -1, -1
    
    #calculate retrieval error CORRELATION matrix  
    Cor=np.zeros((n_dim,n_dim))
    for xx in range(n_dim):
        for yy in range(n_dim):
            Cor[xx,yy]=(S_hat[xx,yy]*S_hat[xx,yy]) / (S_hat[xx,xx]*S_hat[yy,yy])
#    print(np.round(Cor,3))
    
    SIC = 0.5*np.log(np.linalg.det(np.matmul((KtSK+ap_i),ap))) #calculate Shannon Information Content    
    AvgK = np.matmul(S_hat,KtSK) #averaging kernel
    DFS = np.trace(AvgK) #degrees of freedom for signal (DFS) which is trace of averaging kernel
    
    return S_hat, SIC, AvgK, DFS, Cor  #returns retrieval error covariance matrix, the Shannon Information Content,
                        #the averaging kernel, the degrees of freedom for signal, error correlation matrix

In [3]:
#calculates the probability of detection given the parameter value (mu) and uncertainty (sigma)
#assumes PDF is gaussian normally distributed
def detect_prob(mu, sigma, doprint=0): 

    Pd = 1-0.5*(1+erf((-1*mu)/(sigma*np.sqrt(2))))  #detection probability, modified from CDF function

    Pd_pcnt_str=str(np.around(Pd*100,decimals=1))+'% positive probability' #string output version

    if doprint > 0:
        print(Pd_pcnt_str)

    return Pd, Pd_pcnt_str

In [4]:
#calculate detection probability metrics for a full range of fractional plastic coverage. 
#Also, return result at 95% or whatever is specified in fraction_threshold variable
def detect_prob_all(plastic_uncertainty,plastic_fraction,fraction_threshold=0.95):    

    #make array of values to assess (val) and dummy array to fill (det_prob)
    inc=np.arange(0, 10000, 1)
    val=inc/10000
    det_prob=np.arange(0, 10000, 1) / 10000
       
    #interpolate plastic_uncertainty to assessment values
    plastic_uncertainty_int = np.interp(val,plastic_fraction,plastic_uncertainty)    
    
    for x in inc:
        Pd, Pd_pcnt_str = detect_prob(val[x], plastic_uncertainty_int[x], doprint=0)
        det_prob[x] = Pd

    #get plastic fraction for a fraction_threshold detection probability 
    fraction_meeting_threshold = np.interp(fraction_threshold,det_prob,val)
    
    return det_prob, fraction_meeting_threshold
            

In [5]:
def print_out(S_hat, SIC, AvgK, DFS, jac, err, ap, me_err, numpts, params, me_params ):

    S_hat_diag=np.diagonal(S_hat)
    uncert=np.sqrt(S_hat_diag)

    np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
    print('Error covariance matrix:')
    print(S_hat)
    print()

    np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
    print('Averaging kernel matrix:')
    print(AvgK)
    print()
    np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
    print('Model Parameters:       ', params)
    print('Number of observations: ', numpts)
    print('A priori uncertainty:   ', np.sqrt(ap))
    print('Uncertainties:          ', uncert)
    print('Shannon Information Content:      ', SIC)
    print('Degrees of freedom for signal:    ', DFS)

## Section to read simulated dataset

In [27]:
#input/output and other specifics

#file_in='simulations/Amir/plastics_toa_simulations_modisa_permutations_v5_non_biofouled.nc'

rel_err = np.array([0.01587401, 0.01244266, 0.00938955, 0.01092051, 0.00302288,
    0.00544271, 0.00999068, 0.01467843, 0.0080387, 0.00944394,
    0.0193447, 0.0224503, 0.02386379])

######################################################################################################
#file_in='simulations/Amir/plastics_toa_simulations_modisa_permutations_v4.nc' #biofouled case
#doBFunc = True #set to True to include biofouling model uncertainty as defined below 
#outname='data/SQOOP_Amir_BF' #Biofouled, with both model and measurement uncertainty

######################################################################################################
#file_in='simulations/Amir/plastics_toa_simulations_modisa_permutations_v4.nc' #biofouled case
#doBFunc = True #set to True to include biofouling model uncertainty as defined below 
#rel_err = rel_err*2
#outname='data/SQOOP_Amir_BF_x2unc' #Biofouled, with both model and doubled measurement uncertainty

######################################################################################################
#file_in='simulations/Amir/plastics_toa_simulations_modisa_permutations_v4.nc' #biofouled case
#doBFunc = False #set to True to include biofouling model uncertainty as defined below 
#outname='data/SQOOP_Amir_BF_nomodunc' #Biofouled, with measurement uncertainty only (no modeling uncertainty)

######################################################################################################
file_in='simulations/Amir/plastics_toa_simulations_modisa_permutations_v5_non_biofouled.nc' #biofouled case
doBFunc = True #set to True to include biofouling model uncertainty as defined below 
outname='data/SQOOP_Amir_NB' #non-Biofouled, with both model and measurement uncertainty


In [28]:
#section to set up systematic unc based on biofouling difference

if doBFunc:
    meas_BF,meas_NB = pd.read_pickle('data/SQOOP_Amir_biofoul.pkl')

        #systemmatic uncertainty from biofouling is the difference between biofouled and
        #non-biofouled, divided by 6 to represent a 1 sigma difference assuming 99% (3sigma)
        #are represented by that difference between biofouled and non-biofouled
    biofoul_err = (np.abs(meas_BF - meas_NB)) / 6.0
    print('including Biofouling model uncertainty')
else:
    print('NOT including Biofouling model uncertainty')


including Biofouling model uncertainty


In [29]:
#read netcdf4 file with simulation
f = netCDF4.Dataset(file_in)

#read jacobian and data into jac and meas, respectively. Also get z (#cases,), m (measurent), n (param) lengths
jac_all=np.asarray(f.variables['K_Jac'])
meas=np.asarray(f.variables['rhot'])
waveln=np.asarray(f.variables['wavelength'])
g=jac_all.shape
z_len=g[0]
m_len=g[1]
n_len=g[2]

#read the simulation specific parameters into a dataframe
df = pd.DataFrame({'Windspeed(m_s)': np.asarray(f.variables['Windspeed(m_s)']),
                   'Humidity(%)': np.asarray(f.variables['Humidity(%)']),
                   'FMF': np.asarray(f.variables['FMF']),
                   'AOD(869)': np.asarray(f.variables['AOD(869)']),
                   'chla(mg_m3)': np.asarray(f.variables['chla(mg_m3)']),
                   'plastic_fraction': np.asarray(f.variables['plastic_fraction']), 
                   'solz': np.asarray(f.variables['solz']),     
                   'relaz': np.asarray(f.variables['relaz']),  
                   'senz': np.asarray(f.variables['senz']),  
                  })

df["plastic_uncertainty"] = np.nan
df["SIC"] = np.nan
df["plastic_avgK"] = np.nan
df["DFS"] = np.nan
df["Cor_5_0"] = np.nan
df["Cor_5_1"] = np.nan
df["Cor_5_2"] = np.nan
df["Cor_5_3"] = np.nan
df["Cor_5_4"] = np.nan

#close netcdf file
f.close()

In [30]:
#make a priori covariance matrix

#ap=(np.linspace(10.0,10.0,4))**2 #generate a priori error covariance matrix
WS_range=df['Windspeed(m_s)'].max() - df['Windspeed(m_s)'].min()
RH_range=df['Humidity(%)'].max() - df['Humidity(%)'].min()
FMF_range=df['FMF'].max() - df['FMF'].min()
AOD_range=df['AOD(869)'].max() - df['AOD(869)'].min()
CHL_range=df['chla(mg_m3)'].max() - df['chla(mg_m3)'].min()
PF_range=df['plastic_fraction'].max() - df['plastic_fraction'].min()


ap=np.asarray([WS_range,RH_range,FMF_range,AOD_range,CHL_range,PF_range]) #generate a priori error covariance matrix
ap=(ap/2)**2 #generate a priori error covariance matrix diagonals

In [31]:
#make error covariance matrix and prepare jacobian
for idx in range(0, z_len):
#for idx in range(0, 6):

    if (idx % 10000) == 0:
        txt=str(idx)+' of '+str(z_len)
        print(txt)
    
    if doBFunc:
        sys_err = biofoul_err[idx,:]
    else:
        sys_err = np.zeros(13)    
    
    
    err=((meas[idx]*rel_err) + sys_err)**2 #generate error covariance matrix diagonals (code also takes 2d input)        
    
    #prepare jacobian
    this_jac=jac_all[idx]
    jac=this_jac.transpose()

    #check for NaN in jacobian
    if np.isnan(this_jac).any(): 
        print('Nan found in index ',idx)
    if np.isinf(this_jac).any(): 
        print('Inf found in index ',idx)
    
    #calculate rodgers stuff
    S_hat, SIC, AvgK, DFS, Cor = rodgers(jac, err, ap)

    df.loc[idx]["plastic_uncertainty"]=np.sqrt(S_hat[5,5]) 
    df.loc[idx]["SIC"]=SIC
    df.loc[idx]["plastic_avgK"]=AvgK[5,5]
    df.loc[idx]["DFS"]=DFS
    df.loc[idx]["Cor_5_0"]=Cor[5,0] #these are correlation for plastic fraction with: wind speed
    df.loc[idx]["Cor_5_1"]=Cor[5,1] #these are correlation for plastic fraction with: relative humidity
    df.loc[idx]["Cor_5_2"]=Cor[5,2] #these are correlation for plastic fraction with: fine mode fraction
    df.loc[idx]["Cor_5_3"]=Cor[5,3] #these are correlation for plastic fraction with: AOD
    df.loc[idx]["Cor_5_4"]=Cor[5,4] #these are correlation for plastic fraction with: Chl-a           

df_orig=df.copy()    

df

0 of 131220
10000 of 131220
20000 of 131220
30000 of 131220
40000 of 131220
50000 of 131220
60000 of 131220
70000 of 131220
80000 of 131220
90000 of 131220
100000 of 131220
110000 of 131220
120000 of 131220
130000 of 131220


,Windspeed(m_s),Humidity(%),FMF,AOD(869),chla(mg_m3),plastic_fraction,solz,relaz,senz,plastic_uncertainty,SIC,plastic_avgK,DFS,Cor_5_0,Cor_5_1,Cor_5_2,Cor_5_3,Cor_5_4
0,0.5,30.1,0.01,0.04,0.05,0.0001,15.0,40.0,15.0,0.037408,17.651534,0.994401,3.803067,0.957476,0.000134,0.000043,0.787578,0.161159
1,0.5,30.1,0.01,0.04,0.05,0.0001,15.0,40.0,30.0,0.021499,15.349779,0.998151,3.222553,0.952107,0.000026,0.008617,0.999638,0.705399
2,0.5,30.1,0.01,0.04,0.05,0.0001,15.0,40.0,60.0,0.012907,14.811915,0.999334,3.808128,0.153407,0.000454,0.063871,0.996645,0.102897
3,0.5,30.1,0.01,0.04,0.05,0.0001,15.0,110.0,15.0,0.010706,16.276321,0.999541,3.946247,0.199036,0.003501,0.004500,0.992911,0.076764
4,0.5,30.1,0.01,0.04,0.05,0.0001,15.0,110.0,30.0,0.014891,15.591507,0.999113,3.681071,0.878175,0.004009,0.001104,0.998677,0.354362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131215,10.0,94.9,0.95,0.30,2.00,1.0000,60.0,110.0,30.0,0.292222,17.217250,0.658357,3.364043,0.001699,0.984358,0.404711,0.276835,0.001209
131216,10.0,94.9,0.95,0.30,2.00,1.0000,60.0,110.0,60.0,0.046277,6.386301,0.991432,2.862484,0.026467,0.213448,0.000659,0.680919,0.005204
131217,10.0,94.9,0.95,0.30,2.00,1.0000,60.0,170.0,15.0,0.023525,6.282039,0.997786,2.742566,0.002569,0.371079,0.000137,0.713541,0.005944
131218,10.0,94.9,0.95,0.30,2.00,1.0000,60.0,170.0,30.0,0.022669,6.319685,0.997944,2.774965,0.005709,0.151251,0.000718,0.723135,0.010173


In [32]:
#iterate through each set of conditions that have all the same parameter value except for plastic fraction. 
#create a new dataframe, and save one row for each set, with median values for plastic_uncertainty, SIC, plastic_avgK and DFS
#also, assess detection probability and save the plastic fraction value for 90% confident probability.

df.sort_values(by=['Windspeed(m_s)','Humidity(%)','FMF','AOD(869)','chla(mg_m3)','solz','relaz','senz'], inplace=True)

df.rename(columns={'plastic_fraction': 'plastic_threshold', 'plastic_uncertainty': 'plastic_unc_median', 'SIC': 'SIC_median', \
                   'plastic_avgK':'plastic_avgK_median', 'DFS': 'DFS_median', 'Cor_5_0': 'Cor_5_0_median', \
                   'Cor_5_1': 'Cor_5_1_median','Cor_5_2': 'Cor_5_2_median','Cor_5_3': 'Cor_5_3_median', \
                   'Cor_5_4': 'Cor_5_4_median'}, inplace=True)

fdf = df.copy()
fdfe = fdf[0:0]

for idx in range(0, z_len-10, 10):
    if (idx % 10000) == 0:
        txt=str(idx)+' of '+str(z_len)
        print(txt)
    
    #get start and end points for this iteration
    st=idx
    ed=idx+10

    this=df.iloc[st:ed]

    this_plastic_fraction = np.asarray(df.iloc[st:ed]['plastic_threshold'])   
    this_plastic_uncertainty = np.asarray(df.iloc[st:ed]['plastic_unc_median'])   

    this_detect_prob, fraction_meeting_threshold = \
        detect_prob_all(this_plastic_uncertainty,this_plastic_fraction,fraction_threshold=0.95)

    fdfe = fdfe.append(df.iloc[st], ignore_index = True)
        
    fdfe.loc[fdfe.index[-1], 'plastic_threshold']= fraction_meeting_threshold
    
    #section to update field with median values for set
    fdfe.loc[fdfe.index[-1], 'plastic_unc_median']= np.median(this_plastic_uncertainty)
    fdfe.loc[fdfe.index[-1], 'SIC_median']= np.median(np.asarray(df.iloc[st:ed]['SIC_median']))
    fdfe.loc[fdfe.index[-1], 'plastic_avgK_median']= np.median(np.asarray(df.iloc[st:ed]['plastic_avgK_median']))
    fdfe.loc[fdfe.index[-1], 'DFS_median']= np.median(np.asarray(df.iloc[st:ed]['DFS_median']))
    fdfe.loc[fdfe.index[-1], 'Cor_5_0_median']= np.median(np.asarray(df.iloc[st:ed]['Cor_5_0_median']))
    fdfe.loc[fdfe.index[-1], 'Cor_5_1_median']= np.median(np.asarray(df.iloc[st:ed]['Cor_5_1_median']))
    fdfe.loc[fdfe.index[-1], 'Cor_5_2_median']= np.median(np.asarray(df.iloc[st:ed]['Cor_5_2_median']))
    fdfe.loc[fdfe.index[-1], 'Cor_5_3_median']= np.median(np.asarray(df.iloc[st:ed]['Cor_5_3_median']))
    fdfe.loc[fdfe.index[-1], 'Cor_5_4_median']= np.median(np.asarray(df.iloc[st:ed]['Cor_5_4_median']))
    
#change names in original df back
df.rename(columns={'plastic_threshold':'plastic_fraction', 'plastic_unc_median':'plastic_uncertainty', 'SIC_median':'SIC', \
                   'plastic_avgK_median':'plastic_avgK', 'DFS_median':'DFS', 'Cor_5_0_median':'Cor_5_0', \
                   'Cor_5_1_median':'Cor_5_1', 'Cor_5_2_median':'Cor_5_2', 'Cor_5_3_median':'Cor_5_3', \
                   'Cor_5_4_median':'Cor_5_4'}, inplace=True)

fdfe

0 of 131220
10000 of 131220
20000 of 131220
30000 of 131220
40000 of 131220
50000 of 131220
60000 of 131220
70000 of 131220
80000 of 131220
90000 of 131220
100000 of 131220
110000 of 131220
120000 of 131220
130000 of 131220


,Windspeed(m_s),Humidity(%),FMF,AOD(869),chla(mg_m3),plastic_threshold,solz,relaz,senz,plastic_unc_median,SIC_median,plastic_avgK_median,DFS_median,Cor_5_0_median,Cor_5_1_median,Cor_5_2_median,Cor_5_3_median,Cor_5_4_median
0,0.5,30.1,0.01,0.04,0.05,0.089223,15.0,40.0,15.0,0.039837,17.355668,0.993646,3.777812,0.954880,0.000139,0.000043,0.784545,0.155528
1,0.5,30.1,0.01,0.04,0.05,0.039451,15.0,40.0,30.0,0.022180,14.579890,0.998032,3.111027,0.930775,0.000024,0.006143,0.999400,0.690657
2,0.5,30.1,0.01,0.04,0.05,0.046912,15.0,40.0,60.0,0.015110,14.007116,0.999087,3.716834,0.133792,0.000374,0.035695,0.996790,0.177032
3,0.5,30.1,0.01,0.04,0.05,0.035650,15.0,110.0,15.0,0.014433,15.429483,0.999154,3.895116,0.213624,0.001815,0.001816,0.993571,0.096727
4,0.5,30.1,0.01,0.04,0.05,0.034466,15.0,110.0,30.0,0.017816,14.782096,0.998725,3.504769,0.872795,0.001255,0.000781,0.998149,0.357345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13116,10.0,94.9,0.95,0.30,2.00,0.014921,60.0,110.0,15.0,0.011916,10.696753,0.999432,3.263153,0.054997,0.171041,0.002241,0.086587,0.006119
13117,10.0,94.9,0.95,0.30,2.00,0.014592,60.0,110.0,30.0,0.012943,17.621763,0.999330,3.404185,0.023241,0.315354,0.007821,0.183567,0.008409
13118,10.0,94.9,0.95,0.30,2.00,0.068010,60.0,110.0,60.0,0.024123,10.091490,0.997603,3.204446,0.055207,0.298791,0.000603,0.090963,0.009895
13119,10.0,94.9,0.95,0.30,2.00,0.018722,60.0,170.0,15.0,0.011439,10.715292,0.999476,3.321224,0.023740,0.472429,0.000407,0.305049,0.004365


In [33]:
# save dataframes to pickle files

df_name=outname+'_df.pkl'
fdfe_name=outname+'_fdfe.pkl'
vars_name=outname+'_vars.pkl'

df.to_pickle(df_name)
fdfe.to_pickle(fdfe_name) 

f = open(vars_name, 'wb')
pickle.dump([waveln,meas,rel_err,sys_err,jac_all],f)
f.close()